In [1]:
# !pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 -f https://download.pytorch.org/whl/torch_stable.html

In [2]:
# Cloner le dépôt officiel
# !git clone https://github.com/gregzanotti/dlsa-public.git

# Installer les dépendances
# %cd dlsa-public
# !pip install -r requirements.txt

In [ ]:
# TODO: Commenter les lignes contenant 'plt' dans dlsa-public

In [3]:
# !pip install yfinance

In [4]:
import torch


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device utilisé : {device}")

# free GPU memory
torch.cuda.empty_cache()

Device utilisé : cuda


In [5]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

In [6]:
repo_path = 'dlsa-public/'

In [7]:
sys.path.append(repo_path)

In [8]:
from models.CNNTransformer import CNNTransformer
from models.FourierFFN import FourierFFN
from models.OUFFN import OUFFN
from preprocess import preprocess_cumsum, preprocess_fourier, preprocess_ou
from train_test import test

In [9]:
# Fonction pour charger les données de résidus à partir du dépôt (si disponibles)
def load_residual_data(gz_path):
    """
    Tente de charger des données résiduelles pré-calculées du dépôt.
    Retourne (data_array, dates_index, resid_series) ou (None, None, None) si non disponible.
    """
    # Chemin du fichier numpy des résidus (exemple avec modèle Fama-French 5 facteurs)
    filepath = gz_path[:-3]  # Enlève l'extension .gz
    if os.path.exists(filepath) or os.path.exists(gz_path):
        # Décompresser si nécessaire
        if not os.path.exists(filepath) and os.path.exists(gz_path):
            import gzip, shutil
            with gzip.open(gz_path, 'rb') as f_in:
                with open(filepath, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
        # Charger les données numpy
        residuals = np.load(filepath).astype(np.float32)
        T = residuals.shape[0]
        # Générer un index de dates pour T jours ouvrés à partir du 01/01/1998 (approximation)
        dates = pd.bdate_range(start="1998-01-01", periods=T)
        # Calculer la série cumulée du premier résidu pour la stratégie OU+Threshold
        if residuals.ndim > 1:
            resid_series = np.cumsum(residuals[:, 0])
        else:
            resid_series = np.cumsum(residuals)
        return residuals, dates, resid_series
    else:
        return None, None, None

In [10]:
# Fonction pour exécuter et évaluer une stratégie deep learning (CNN+Transformer ou Fourier+NN)
def run_deep_strategy(name, model_class, preprocess_func, data, dates, config):
    """
    Entraîne et teste la stratégie de trading spécifiée (model_class avec preprocess_func) sur les données fournies.
    Renvoie un dict contenant les métriques de performance et les rendements journaliers.
    """
    cfg = config.copy()
    # Exécuter la simulation train/test
    rets, sharpe, ret, std, turnover, short_prop = test(
        data,
        dates,
        model_class,
        preprocess_func,
        cfg,
        residual_weights=None,
        save_params=False,
        force_retrain=cfg['force_retrain'],
        parallelize=False,
        device='cuda',
        output_path=".",
        log_dev_progress_freq=50,
        log_plot_freq=50,
        num_epochs=cfg['num_epochs'],
        early_stopping=cfg['early_stopping'],
        batchsize=cfg['batch_size'],
        retrain_freq=cfg['retrain_freq'],
        rolling_retrain=cfg['rolling_retrain'],
        length_training=cfg['length_training'],
        lookback=cfg['model']['lookback'],
        trans_cost=cfg['trans_cost'],
        hold_cost=cfg['hold_cost'],
        objective=cfg['objective'],
        model_tag=name
    )
    # Calculer les métriques
    mean_daily_ret = float(ret)      # rendement moyen quotidien (non annualisé)
    daily_vol = float(std)           # volatilité quotidienne
    daily_sharpe = float(sharpe)     # Sharpe ratio quotidien (mean/std)
    ann_ret = mean_daily_ret * 252.0 if daily_vol != 0 else 0.0
    ann_vol = daily_vol * np.sqrt(252.0)
    ann_sharpe = daily_sharpe * np.sqrt(252.0) if daily_vol != 0 else 0.0
    # Générer et sauvegarder les graphiques
    out_dates = dates[-len(rets):]  # dates correspondantes à la période test
    cum_returns = np.cumprod(1 + rets)
    # Résultats
    return {
        'annual_return': ann_ret,
        'annual_volatility': ann_vol,
        'sharpe_ratio': ann_sharpe,
        'daily_returns': rets,
        'cumulative_returns': cum_returns,
        'dates': out_dates
    }

In [ ]:
# Fonction pour exécuter et évaluer la stratégie OU + Threshold
def run_threshold_strategy(name, resid_series, dates, config, threshold_coef=2.0):
    """
    Simule une stratégie de retour à la moyenne avec seuil sur la série résiduelle fournie.
    Entrée longue si le résidu < -seuil, entrée courte si résidu > +seuil, sortie de position quand le résidu repasse par 0.
    Renvoie un dict contenant les métriques de performance et les rendements journaliers.
    """
    T = len(resid_series)
    train_len = config.get('length_training', int(T * 0.5))
    train_len = min(train_len, T - 1)
    # Calibrer le seuil sur la période d'entraînement
    std_train = float(np.std(resid_series[:train_len]))
    threshold = threshold_coef * std_train
    pos = 0  # position courante (1 = long résidu, -1 = short résidu)
    prev_resid = resid_series[train_len - 1]
    returns_list = []
    for t in range(train_len, T):
        if t > train_len:
            # P&L du jour t (différence du résidu multipliée par la position détenue)
            pnl = pos * (resid_series[t] - prev_resid)
            returns_list.append(pnl)
        # Mettre à jour la position à la fin du jour t
        if pos == 0:
            if resid_series[t] > threshold:
                pos = -1  # résidu haut -> vendre
            elif resid_series[t] < -threshold:
                pos = 1   # résidu bas -> acheter
        elif pos == 1:
            if resid_series[t] >= 0:
                pos = 0   # clôturer la position longue
        elif pos == -1:
            if resid_series[t] <= 0:
                pos = 0   # clôturer la position courte
        prev_resid = resid_series[t]
    returns = np.array(returns_list, dtype=np.float32)
    test_dates = dates[-len(returns):] if len(returns) < len(dates) else dates[train_len+1:]
    # Calcul des métriques
    mean_daily_ret = returns.mean() if returns.size > 0 else 0.0
    daily_vol = returns.std(ddof=0) if returns.size > 0 else 0.0
    daily_sharpe = mean_daily_ret/daily_vol if daily_vol != 0 else 0.0
    ann_ret = mean_daily_ret * 252.0 if daily_vol != 0 else 0.0
    ann_vol = daily_vol * np.sqrt(252.0)
    ann_sharpe = daily_sharpe * np.sqrt(252.0) if daily_vol != 0 else 0.0
    cum_returns = np.cumprod(1 + returns)
    # Résultats
    return {
        'annual_return': ann_ret,
        'annual_volatility': ann_vol,
        'sharpe_ratio': ann_sharpe,
        'daily_returns': returns,
        'cumulative_returns': cum_returns,
        'dates': test_dates
    }

In [13]:
# Charger les données de résidus
residual_data_name = "AvPCA_OOSresiduals_3_factors_1998_initialOOSYear_60_rollingWindow_252_covWindow_0.01_Cap.npy"
gz_path = os.path.join(repo_path, "residuals", "pca", residual_data_name + ".gz")
data, dates, resid_series = load_residual_data(gz_path)

# S'assurer que data est un tableau 2D numpy (T x N)
T = data.shape[0]
if data.ndim == 1:
    data = data.reshape(-1, 1)
N = data.shape[1]
print(f"Nombre de jours de données : {T}, Nombre de séries résiduelles : {N}")

# Définir la longueur de la période d'entraînement initiale (length_training)
length_training = 1000
if T <= length_training:
    length_training = max(int(T * 0.5), 1)
length_training = min(length_training, T - 1)  # au moins 1 jour de test

# Après avoir chargé les résidus
if data.shape[1] > 5000:
    data = data[:, :5000]
    print(f"Réduction : utilisation de seulement {data.shape[1]} résidus.")

Nombre de jours de données : 4781, Nombre de séries résiduelles : 9483
Réduction : utilisation de seulement 5000 résidus.


In [14]:
# Configuration commune pour les stratégies deep learning
config = {
    'model': {'lookback': 30},
    'force_retrain': True,
    'early_stopping': False,
    'rolling_retrain': True,
    'retrain_freq': 250,
    'length_training': length_training,
    'num_epochs': 100,
    'batch_size': 512,
    'optimizer_name': 'Adam',
    'optimizer_opts': {'lr': 0.001},
    'trans_cost': 0.0,
    'hold_cost': 0.0,
    'objective': 'sharpe',
    'mode': 'test',
    'debug': False,
    'results_tag': ''
}

___

In [16]:
print("\nExécution de la stratégie CNN+Transformer...")
strategies_results1 = run_deep_strategy('CNNTransformer', CNNTransformer, preprocess_cumsum, data, dates, config)


Exécution de la stratégie CNN+Transformer...


In [17]:
strategies_results1 = {
    'annual_return': strategies_results1['annual_return'],
    'annual_volatility': strategies_results1['annual_volatility'],
    'sharpe_ratio': strategies_results1['sharpe_ratio'],
    'daily_returns': strategies_results1['daily_returns'].tolist(),
    'cumulative_returns': strategies_results1['cumulative_returns'].tolist(),
    'dates': [str(d).split()[0] for d in strategies_results1['dates']]
}

In [ ]:
# save the json file
import json
with open(f'results/{residual_data_name[:-4]}_CNN+Transformer_results.json', 'w') as f:
    json.dump(strategies_results1, f, indent=4)

___

In [19]:
# Stratégie 2 : Fourier + NN
print("\nExécution de la stratégie Fourier+NN...")
strategies_results2 = run_deep_strategy('FourierFFN', FourierFFN, preprocess_fourier, data, dates, config)


Exécution de la stratégie Fourier+NN...


In [20]:
strategies_results2 = {
    'annual_return': strategies_results2['annual_return'],
    'annual_volatility': strategies_results2['annual_volatility'],
    'sharpe_ratio': strategies_results2['sharpe_ratio'],
    'daily_returns': strategies_results2['daily_returns'].tolist(),
    'cumulative_returns': strategies_results2['cumulative_returns'].tolist(),
    'dates': [str(d).split()[0] for d in strategies_results2['dates']]
}

In [ ]:
# save the json file
import json
with open(f'results/{residual_data_name[:-4]}_Fourier+NN_results.json', 'w') as f:
    json.dump(strategies_results2, f, indent=4)

___